In [47]:
import os
import sys
import numpy as np
from matplotlib import pyplot as plt
import cv2
from PIL import Image
import pandas as pd
import csv 
from sklearn import datasets
import argparse
import joblib 


In [48]:
# import models modules
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn import cluster
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold, StratifiedKFold, train_test_split 
from sklearn.feature_selection import RFECV
from scipy.stats import randint, loguniform


In [49]:
# import stats modules
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [50]:
# import tree visualisation modules
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [51]:
# import general visualisation modules
import plotly.express as px 
import matplotlib.pyplot as plt 

In [52]:
# set printing option to print the whole large variable
np.set_printoptions(threshold=np.inf)
# declaration of cross-validator
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=2, random_state=1)
# declaration of scaler for dataset numerical features
sc = MinMaxScaler()

In [53]:
def churn_causal_features(dataset):

    ## Using the generalised linear model get statistical significance insights on the effect of features on the target (churn)

    # Change variable name separators to '_'
    mod_features = [col.replace(" ", "_").replace("(", "_").replace(")", "_").replace("-", "_").replace("_&_", "_").replace(",_", "_").replace("/", "_") for col in dataset.columns]

    # Effect the change to the dataframe column names
    dataset.columns = mod_features

    # Prepare it for the GLM formula
    GLM_features = [col for col in dataset.columns if col not in ['SALESFORCEACCOUNTID', 'CHURN']]
    GLM_features = ' + '.join(map(str, GLM_features)) # map(func, list)

    # Fit it to GLM model
    GLM_model = smf.glm(formula=f'CHURN ~ {GLM_features}', data=dataset, family=sm.families.Binomial())
    results = GLM_model.fit()

    # Extract features with p-value in abs < 0.05 as the features causing customers to churn or not
    print(results.summary())
    # Insights: division by zero encountered in the log calculation of teh GLM, all p-values are < 0.05 (very small), i.e all features affect the churn rate 

    # Extract the most important features for the churn rate prediction problem, > 1
    print(np.exp(results.params))
    # insights: overflow encountered in exp, inf values reuslt for the features: seats, sector, etc



In [54]:
def features_correlation(dataset):
    corr = dataset.corr()

    fig = px.imshow(corr,width=2000, height=2000)
    fig.show()

In [55]:
#Performance evaluation
def performance_eval(y_true, y_pred):

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')

    return accuracy, precision, recall, f1

In [56]:
# Model Selection and churn prediction
def ChurnPred(X, target,  model= "LR", params={}):
    model = model(**params)

    X_train, X_test, Y_train, Y_test = train_test_split(X, target, test_size=0.3, random_state=50)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    acc, precision, recall, f1 = performance_eval(Y_test, Y_pred)
    return model, acc, precision, recall, f1

In [57]:
def extract_dataset(data_dir, filename):
    ## Load dataset
    Df = pd.read_csv(os.path.join(data_dir, filename) + ".csv")
    #print(Df.head(10))

    ## Extract information from the dataset
    (records_sz, features_sz) = Df.shape
    print("The dataset is composed of %d data entry and %d features" %(records_sz, features_sz))

    ## Features
    features = Df.columns
    #print("Features available are:", features)

    ## Number of customers
    customers = np.unique(Df["SALESFORCEACCOUNTID"])
    #print("unique customers are:", customers)
    nb_customers = len(customers)
    print("Total number of customers in available dataset is:", nb_customers)

    ## Extract number of customers who churned
    customers_churn = []

    for cus in customers:
        df_cus = Df.loc[Df["SALESFORCEACCOUNTID"]==cus]
        customers_churn.append(np.sum(df_cus["CHURN"]))

    print("the max churn rate per customer is =", np.max(customers_churn))

    nb_customers_churn = np.sum(customers_churn)
    print("The number of customers who churned is %d while the number of customers who didn't churn is %d " %(nb_customers_churn, nb_customers-nb_customers_churn))
    print("The percentage of customers who churned is %.02f  while the number of customers who didn't churn is %.02f " %((nb_customers_churn/nb_customers)*100, ((nb_customers-nb_customers_churn)/nb_customers)*100))

    ## Extract the set of features that may have caused the churn to happen
    Df_features = Df.copy(deep=True)
    causal_features = Df_features.drop(["SALESFORCEACCOUNTID", "CHURN", "ACCOUNTING_MONTH", "CONTRACT_START_DATE", "RENEWAL_MONTH"], axis =1).columns

    ## Exploring data types
    print("data types:", Df.dtypes)
    #print("DNB types:", Df["DNB_GLOBAL_SALES_REVENUE"].dtypes)
    #print("DNB types:", Df["DNB_GLOBAL_EMPLOYEE_COUNT"].dtypes)

    print("number of unique feature values:", Df.nunique())

    return Df, nb_customers

In [58]:
def data_processing(df, nb_customers):


    ## processing mumerical features 
    ## processing empty fields: GLOBAL_REVENUE, GLOBAL_EMPLOYEE_COUNT, SURVEY_AVG_CXI_SCORE, SURVEY_AVG_NPS_SCORE, SURVEY_AVG_CASE_MOOD_SCORE
    try:
        df['SURVEY_AVG_CXI_SCORE'] = df['SURVEY_AVG_CXI_SCORE'].astype(float)
    except ValueError as err:
        print (err)


    ## Extract percentages of empty fields for each feature
    # feature 'DNB_GLOBAL_SALES_REVENUE'
    ind_customers_empty_global_rev = df['DNB_GLOBAL_SALES_REVENUE'].index[df['DNB_GLOBAL_SALES_REVENUE'].apply(np.isnan)]
    nb_customers_empty_global_rev = df["SALESFORCEACCOUNTID"][ind_customers_empty_global_rev].nunique()

    print("percentage of customers with empty global revenue is %.02f" %((nb_customers_empty_global_rev/nb_customers)*100))

    # feature 'DNB_GLOBAL_EMPLOYEE_COUNT'
    ind_customers_empty_nemployee = df['DNB_GLOBAL_EMPLOYEE_COUNT'].index[df['DNB_GLOBAL_EMPLOYEE_COUNT'].apply(np.isnan)]
    nb_customers_empty_nemployee = df["SALESFORCEACCOUNTID"][ind_customers_empty_nemployee].nunique()

    print("percentage of customers with empty employee count is %.02f" %((nb_customers_empty_nemployee/nb_customers)*100))

    # feature SURVEY CXI score
    ind_customers_empty_CXI = df['SURVEY_AVG_CXI_SCORE'].index[df['SURVEY_AVG_CXI_SCORE'].apply(np.isnan)]
    nb_empty_CXI = len(ind_customers_empty_CXI)

    print("percentage of empty surveys CXI score is %.02f" %((nb_empty_CXI/len(df))*100))


    # feature SURVEY NPS score
    ind_customers_empty_NPS = df['SURVEY_AVG_NPS_SCORE'].index[df['SURVEY_AVG_NPS_SCORE'].apply(np.isnan)]
    nb_empty_NPS = len(ind_customers_empty_NPS)

    print("percentage of empty surveys NPS score is %.02f" %((nb_empty_NPS/len(df))*100))


    # feature SURVEY mood score
    ind_customers_empty_MOOD = df['SURVEY_AVG_CASE_MOOD_SCORE'].index[df['SURVEY_AVG_CASE_MOOD_SCORE'].apply(np.isnan)]
    nb_empty_MOOD = len(ind_customers_empty_MOOD)

    print("percentage of empty surveys case mood score is %.02f" %((nb_empty_MOOD/len(df))*100))


    ### NB: We Can fill the empty survey scores with the median of scores filled per user, but the number of cutomers who didn't fill the surveys at any time is large so we drop the feature

    dropping_list = ["SALESFORCEACCOUNTID", "ACCOUNTING_MONTH", "CONTRACT_START_DATE", "RENEWAL_MONTH"]
    missing_dropping_list = ["DNB_GLOBAL_SALES_REVENUE", "DNB_GLOBAL_EMPLOYEE_COUNT", "SURVEY_AVG_CXI_SCORE", "SURVEY_AVG_NPS_SCORE", "SURVEY_AVG_CASE_MOOD_SCORE"]
    categorical_features = ["REGION", "SECTOR"]

    rm_features = dropping_list + missing_dropping_list + ["CHURN"] + categorical_features
    
    num_list = df.drop(rm_features, axis =1)

    df = df.drop(dropping_list, axis=1)

    df = df.drop(missing_dropping_list, axis=1)


    ## Feature scaling
    for ft in num_list.columns:
        df[ft] = sc.fit_transform(df[[ft]])

    ## processing multi-categorical features
    df = pd.get_dummies(df, drop_first=True)

    print(df.isnull().sum())

    return df

In [66]:
# Main body

data_dir = os.getcwd()
filename = "dataset"
#args =  parser.parse_args()

dataset_df, nb_customers = extract_dataset(data_dir, filename)
dataset = data_processing(dataset_df, nb_customers)
X = dataset.drop("CHURN", axis =1)
target = dataset["CHURN"]

# correlation between features
features_correlation(dataset)

# Extract statistics
churn_causal_features(dataset)


# Optimal number of features to fit a model
rfecv = RFECV(estimator=RandomForestClassifier(), cv=StratifiedKFold(2, random_state=2, shuffle=True), scoring="accuracy")
rfecv.fit(X, target)
print("The optimal number of features is %d" %(rfecv.n_features_)) # 7

# save the dataframe with optimal features
X_rfe = X.iloc[:, rfecv.support_]

The dataset is composed of 100000 data entry and 82 features
Total number of customers in available dataset is: 3336
the max churn rate per customer is = 1
The number of customers who churned is 670 while the number of customers who didn't churn is 2666 
The percentage of customers who churned is 20.08  while the number of customers who didn't churn is 79.92 
data types: SALESFORCEACCOUNTID            object
ACCOUNTING_MONTH               object
RENEWAL_MONTH                  object
CONTRACT_START_DATE            object
REGION                         object
                               ...   
BACKLOG                         int64
SURVEY_AVG_CXI_SCORE          float64
SURVEY_AVG_NPS_SCORE          float64
SURVEY_AVG_CASE_MOOD_SCORE    float64
CHURN                           int64
Length: 82, dtype: object
number of unique feature values: SALESFORCEACCOUNTID           3336
ACCOUNTING_MONTH                42
RENEWAL_MONTH                   90
CONTRACT_START_DATE           1538
REGION   

/home/sariah/anaconda3/envs/jupyter/lib/python3.7/site-packages/statsmodels/genmod/families/links.py:187: RuntimeWarning:

overflow encountered in exp

/home/sariah/anaconda3/envs/jupyter/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:1014: RuntimeWarning:

divide by zero encountered in log

/home/sariah/anaconda3/envs/jupyter/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:1014: RuntimeWarning:

invalid value encountered in multiply

/home/sariah/anaconda3/envs/jupyter/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

overflow encountered in exp



                 Generalized Linear Model Regression Results                  
Dep. Variable:                  CHURN   No. Observations:               100000
Model:                            GLM   Df Residuals:                    99901
Model Family:                Binomial   Df Model:                           98
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Sun, 26 Nov 2023   Deviance:                       67235.
Time:                        21:52:17   Pearson chi2:                 3.29e+18
No. Iterations:                   100   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                                             coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

The optimal number of features is 7


In [ ]:
# Insight#1: a division by zero is encountered in the log calculation of the GLM, all p-values are < 0.05 (very small), i.e all features are statistically significant to the churn rate 

# Insight#2: overflow encountered in exp(params), inf values are associated to the higher important features: seats, sector, etc

# Insight#3: further investigations need to be done to debug the inf values in the exp(params). Example: investigate into non-linear statistical models
# as SHAPLEY value and MI (mutual information) instead of leveraging on GLM to infer on statistical significance of features and on the most important features


In [62]:
# Churn prediction: models comparision

churn_pred_1, acc, precision, recall, f1 = ChurnPred(X, target, model = LogisticRegression)
print("The model %s performance is evaluated as having accuracy = %.02f, precision = %.02f, recall = %.02f, and f1 score = %.02f " %(str(churn_pred_1), acc, precision, recall, f1))

## Compare performance to other models 
# We could compare to deep net but we have class imbalance in Churn prediction dataset

churn_pred_2, acc, precision, recall, f1 = ChurnPred(X, target, model = SVC)
print("The model %s performance is evaluated as having accuracy = %.02f, precision = %.02f, recall = %.02f, and f1 score = %.02f " %(str(churn_pred_2), acc, precision, recall, f1))

churn_pred_3, acc, precision, recall, f1 = ChurnPred(X, target, model = GaussianNB)
print("The model %s performance is evaluated as having accuracy = %.02f, precision = %.02f, recall = %.02f, and f1 score = %.02f " %(str(churn_pred_3), acc, precision, recall, f1))

churn_pred_4_ , acc, precision, recall, f1 = ChurnPred(X_rfe, target, model = RandomForestClassifier) # accuracy = 1.00, precision = 0.99, recall = 1.00, and f1 score = 1.00
print("The model %s performance with feature selection is evaluated as having accuracy = %.02f, precision = %.02f, recall = %.02f, and f1 score = %.02f " %(str(churn_pred_4_), acc, precision, recall, f1))
churn_pred_4, acc, precision, recall, f1 = ChurnPred(X, target, model = RandomForestClassifier) # accuracy = 1.00, precision = 0.98, recall = 0.99, and f1 score = 1.00 
print("The model %s performance is evaluated as having accuracy = %.02f, precision = %.02f, recall = %.02f, and f1 score = %.02f " %(str(churn_pred_4), acc, precision, recall, f1))

churn_pred_5, acc, precision, recall, f1 = ChurnPred(X, target, model = DecisionTreeClassifier)
print("The model %s performance is evaluated as having accuracy = %.02f, precision = %.02f, recall = %.02f, and f1 score = %.02f " %(str(churn_pred_5), acc, precision, recall, f1))

churn_pred_6, acc, precision, recall, f1 = ChurnPred(X, target, model = GradientBoostingClassifier)
print("The model %s performance is evaluated as having accuracy = %.02f, precision = %.02f, recall = %.02f, and f1 score = %.02f " %(str(churn_pred_6), acc, precision, recall, f1))

The model LogisticRegression() performance is evaluated as having accuracy = 0.99, precision = 0.55, recall = 0.06, and f1 score = 0.99 


/home/sariah/anaconda3/envs/jupyter/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



The model SVC() performance is evaluated as having accuracy = 0.99, precision = 0.00, recall = 0.00, and f1 score = 0.99 
The model GaussianNB() performance is evaluated as having accuracy = 0.97, precision = 0.16, recall = 0.98, and f1 score = 0.98 
The model RandomForestClassifier() performance with feature selection is evaluated as having accuracy = 1.00, precision = 0.98, recall = 0.99, and f1 score = 1.00 
The model RandomForestClassifier() performance is evaluated as having accuracy = 1.00, precision = 0.98, recall = 0.99, and f1 score = 1.00 
The model DecisionTreeClassifier() performance is evaluated as having accuracy = 1.00, precision = 0.99, recall = 0.98, and f1 score = 1.00 
The model GradientBoostingClassifier() performance is evaluated as having accuracy = 1.00, precision = 0.99, recall = 0.95, and f1 score = 1.00 


In [63]:
## Insights on models comparison
# Based on the f1 score of all 5 models evaluated, we narrow down the comparison between DecisionTreeClassifier() and RandomForestClassifier()
# Among those 2, since recall (FN) is more important for the churn prediction study than precision, 
# we pick the RandomForestClassifier() model as best among the 6


In [ ]:
# Hyper-parameters optimisation for LR model
search_space = dict()
search_space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
search_space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
search_space['C'] = loguniform(1e-5, 100)

## Randomized search on hyper parameters
params_search = RandomizedSearchCV(LogisticRegression(), search_space, n_iter=10, scoring='accuracy', n_jobs=-1, cv=cv, random_state=1)

search_result = params_search.fit(X, target)
best_params = search_result.best_params_

model_tuned, _, _, _, _ = ChurnPred(X, target, model = LogisticRegression, params=best_params)

In [65]:
## save best model
filename = "best_model_churn.sav"
joblib(churn_pred_4_, filename)

TypeError: 'module' object is not callable